# CS380 UNet

**Author:** Will Hewitt

### Purpose

Trains UNet for segmenting echocardiography images

### Conda Environment

`conda activate heartlab`

### Data Sources

`/data/camus/v5/{x_train/test/val | y_train/test/val}.npy`

### References

CS380-Prepare-Data.ipynb, CS380-UNet-Evaluate

In [1]:
import sys
import datetime

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, Concatenate

sys.path.append('../segmentation_utils')

from segmentation_utils import jaccard_distance_loss, dice_loss

Using TensorFlow backend.


###### Import Dataset

In [2]:
path = "/srv/jupyterhub_notebooks/data/camus/v5/"

x_train, x_val = np.load(path + 'x_train.npy'), np.load(path + 'x_val.npy')
y_train, y_val = np.load(path + 'y_train.npy'), np.load(path + 'y_val.npy')

###### Hyperparameters

Make sure to recompile the whole model after changing these

In [3]:
bottleneck_dropout_thresh = 0.5
adam_optimizer = Adam(learning_rate=1e-5)
freeze_encoder_weights = False # If True, encoder weights ARE frozen. If False, encoder weights ARE NOT frozen.
n_epochs=50
batch_size=16
loss_function='categorical_crossentropy'

###### Build Encoder

In [4]:
input_layer = Input(shape=(256,256,1))
input_concat = Concatenate()([input_layer, input_layer, input_layer])

encoder_model = VGG16(input_tensor=input_concat, include_top=False, weights='imagenet', pooling=None)

# Cut off the last conv block

encoder_model = Model(inputs=input_layer, outputs=encoder_model.get_layer("block4_pool").output)

# Set weights frozen

for layer in encoder_model.layers:
    layer.trainable = !freeze_encoder_weights

###### Build Bottleneck and Decoder

In [5]:
# bottleneck

bottleneck1_1 = Dropout(bottleneck_dropout_thresh)(Conv2D(1024, (3,3), activation='relu', padding='same')(encoder_model.output))
bottleneck1_2 = Dropout(bottleneck_dropout_thresh)(Conv2D(1024, (3,3), activation='relu', padding='same')(bottleneck1_1))

# up block 1

upconv1_1 = Conv2DTranspose(512, 2, (2,2), padding='same', activation='relu')(bottleneck1_2)
upconv1_2 = Concatenate(axis=3)(inputs=[upconv1_1, encoder_model.get_layer(name='block4_conv3').output])
upconv1_3 = Conv2D(512, (3,3), activation='relu', padding='same')(upconv1_2)
upconv1_4 = Conv2D(512, (3,3), activation='relu', padding='same')(upconv1_3)

# up block 2

upconv2_1 = Conv2DTranspose(256, 2, (2,2), padding='same', activation='relu')(upconv1_4)
upconv2_2 = Concatenate(axis=3)(inputs=[upconv2_1, encoder_model.get_layer(name='block3_conv3').output])
upconv2_3 = Conv2D(256, (3,3), activation='relu', padding='same')(upconv2_2)
upconv2_4 = Conv2D(256, (3,3), activation='relu', padding='same')(upconv2_3)

# up block 3

upconv3_1 = Conv2DTranspose(128, 2, (2,2), padding='same', activation='relu')(upconv2_4)
upconv3_2 = Concatenate(axis=3)(inputs=[upconv3_1, encoder_model.get_layer(name='block2_conv2').output])
upconv3_3 = Conv2D(128, (3,3), activation='relu', padding='same')(upconv3_2)
upconv3_4 = Conv2D(128, (3,3), activation='relu', padding='same')(upconv3_3)

# up block 4

upconv4_1 = Conv2DTranspose(64, 2, (2,2), padding='same', activation='relu')(upconv3_4)
upconv4_2 = Concatenate(axis=3)(inputs=[upconv4_1, encoder_model.get_layer(name='block1_conv2').output])
upconv4_3 = Conv2D(64, (3,3), activation='relu', padding='same')(upconv4_2)
upconv4_4 = Conv2D(64, (3,3), activation='relu', padding='same')(upconv4_3)

# output layer
output_layer = Conv2D(4, (1,1), activation='softmax')(upconv4_4)

###### Setup TensorBoard logging

In [6]:
run_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + run_name

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

###### Compile final model

In [7]:
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=adam_optimizer, 
              loss=loss_function, 
              metrics=[MeanIoU(num_classes=4)])

###### Train model

In [8]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback]
)

model.save('models/' + run_name + '.h5')

Train on 720 samples, validate on 90 samples
Epoch 1/50
720/720 [==============================] - 28s 39ms/sample - loss: 1.3378 - mean_io_u: 0.3750 - val_loss: 0.8280 - val_mean_io_u: 0.3750
Epoch 2/50
720/720 [==============================] - 20s 27ms/sample - loss: 0.7036 - mean_io_u: 0.3750 - val_loss: 0.5582 - val_mean_io_u: 0.3751
Epoch 3/50
720/720 [==============================] - 20s 27ms/sample - loss: 0.4892 - mean_io_u: 0.3755 - val_loss: 0.4343 - val_mean_io_u: 0.3758
Epoch 4/50
720/720 [==============================] - 20s 27ms/sample - loss: 0.4038 - mean_io_u: 0.3773 - val_loss: 0.3756 - val_mean_io_u: 0.3763
Epoch 5/50
720/720 [==============================] - 20s 27ms/sample - loss: 0.3417 - mean_io_u: 0.3801 - val_loss: 0.3115 - val_mean_io_u: 0.3778
Epoch 6/50
720/720 [==============================] - 20s 27ms/sample - loss: 0.2888 - mean_io_u: 0.3856 - val_loss: 0.3201 - val_mean_io_u: 0.3780
Epoch 7/50
720/720 [==============================] - 20s 27ms/samp